# Tissue Atlas Setup Walkthrough

Tutorial author: Lucas Steinberger

This notebook walks a user through the process of setting up the DKFZ htc framework to work data from the DKFZ tissue atlas dataset. The Tissue Atlas dataset is organized differently from the data used by the original htc framework, so a slightly edited version of the htc framework must me used. This version retains the ability to use default datasets from the original htc framework (or aims to), but also allows users to access the tissue atlas.

This notebook will guide the user through the initial setup necessary to use the framework. After completing this notebook, users can proceed with either 'TissueAtlassClassTraining.ipynb' to perform classification on pre-segmented organs, or with 'SegmentationTraining.ipynb' to train an organ segmentation model. The notebook is designed for use by a medical researcher with little to no python experience, and it will walk you explicitly through all the steps you need to get started and run your own training and inference on a dataset of your choice.

Reminder: htc is only executable on Ubuntu. If you are running windows, I recommend using WSL to get set up with linux capabilities.


## PATH Environment Variables
In order to use the htc framework, you must first define PATH Environment variables in an appropriate file. Please consult the README in the htc repository for a more detailed treatment of these environment variables. Here, we will provide a basic overview of how they work and instructions on how to use them.

PATH environment variables are variables that the htc framework uses to locate the dataset(s) that you want it to use. First, you must create an environments file. If you have cloned the htc framework as suggested in the readme, simply navigate to the repositories root Directory (should be named htc). In this directory, create a new file named ".env"

For example, in a bash terminal, run:

```bash
 cd ~/path/to/my/htc
 nano .env
```

Replace "~/path/to/my/htc" with the actual path on your system to the cloned htc repository.

Now, you must define your PATH variables. the htc framework uses a specific naming convention, so please follow the steps carefully.

### PATH to dataset(s):
For our purposes, there are three types of PATH environment variables used by the framework. The first is the PATH_Tivita variable, which tells the framework where to look for the dataset(s) that you wish to use. In the current iteration of the framework, a 'dataset" has a specific meaning: a dataset is a directory in the larger Tissue Atlas folder structure, that itself contains a directory titled "data", which is where the imaging data itself is actually kept. (while this may seem arbitrary to the user, it is essential for the code to run!!)


Importantly, the "dataset" path should not be to the "data" directory itself, it MUST point to a directory that is theparent of a "data" directory!!

For example the following path is a valid path to a dataset:

```bash
 ~/TIVITA_Cat/Cat_Pig/Cat_atlas/Cat_0002_small_bowel
```
However, the next two are NOT valid "dataset" paths:

```bash
 ~/TIVITA_Cat/Cat_Pig/Cat_atlas/Cat_0002_small_bowel/data
 ~/TIVITA_Cat/Cat_Pig/Cat_atlas
```

Once you have found your dataset path, you can define a Path environment variable by copying and pasting and the following code into the .env file. (replacING the paths with your own path)
```bash
 PATH_Tivita_Cat_0002_small_bowel="~/TIVITA_Cat/Cat_Pig/Cat_atlas/Cat_0002_small_bowel:shortcut=smallbowel"
```
The shortcut here provides an extra way to access your dataset further in the framework, name it how you like. However, the variable name is NOT trivial. it must follow the form:
```bash
 PATH_Tivita_<your_dataset_directory_name>
```
If the variable name does not match the path it is handed, the framework will not work.

If you have multiple datasets that you would like to use, simply add them in the same way to your .env folder. Finally, when you are done, in the root of your htc directory run:
```bash
source .env
```
You can also copy this source line to your .bashrc, so that every time you open a new terminal the .env file is automatically sourced

## PATH to Results

You also need to add a path to a "Results" folder. this is where the framework will send the output of training or inference tasks. the setup is pretty much the same as with the dataset, except simpler:
```bash
 PATH_HTC_RESULTS="~/path/to/results"
```
Do not specify a shortcut for the results path. You don't need one, and the framework will not recognize it. If you want to have multiple results folders, you can do that as well, for instructions please consult the README.md in the htc repository

## PATH to External

The original htc framework is designed and strongly integrated around an expected dataset structure. In particular, The original htc expects a dataset to be divided into "intermediates" and "data", and within the "data" folder to find a specific json named "dataset_settings.json" (more on that later). However, with the tissue atlas we are somewhat stuck with the structure we have, and we cannot go adding folders and jsons to the existing structure.

Instead, this modified framework allows you to specify an extra PATH environment variable to a directory named "external". This directory is in many ways interpreted by the framework as a dataset, and should mimic the expected structure of the dataset (with intermediates and data subdirectories). However, it contains no core data, and instead houses all the extra metadata, preprocessed data, and configuration files needed by the framework. The modified framework is designed to be run with EXACTLY ONE ACTIVE EXTERNAL DIRECTORY at a time, and it is essentail to the functions of the codebase.

If this is confusing, Don't worry -- all you should know  to use it is that the externals directory is a workaround to allow us to build our extra files and information in a location of our choosing, that is not relative to the dataset. 

To define the variable, create an "external" directory at your desired location of your choosing on your system. Then write in .env:

```bash
 PATH_HTC_EXTERNAL="~/path/to/external:shortcut=external"
```
Again, you may create a shortcut of your choosing, but I recommend using simply "external"




# Building and filling your external directory

Now that we have defined an external directory, we need to make sure it has all the right contents. This consists of creating the "data" and "intermediates" subdirectories, and populating them with the appropriate files. 

Generally, the modified codebase is designed to have one external directory per experiment/analysis project. you may always create as many external directories as you likeas long as you only have one sourced from your .env at a time. However, the directory can get large, so due to space reasons you may want to delete old external directoriies when you are done with them.

At this point, you should have your datasets, results folder, and external folder all defined in your .env. The last big step in setup is to define your label mapping.

The Label Mapping is what the codebase uses to read and assign labels to your annotations (and in the case of segmentations, to its own predicted segmentation masks.) There are actually two times different times a label mapping gets used: once in the setup, when creating the contents of the external directory, and then again later when analyzing the data. the second label mapping will be adressed in further tutorials. For now, we will adress the first case.

You can define your label mapping with a JSON file titled "subdata_hypergui_mapping.json". An example is given below:

In [2]:
{
	"Cat_0002_small_bowel#_hypergui_1": 
		{"label_name" : "small_bowel",
		"annotation_type" : "polygon",
		"annotator" : "annotator1"},
  "Cat_0003_colon#_hypergui_1":
    {"label_name" : "colon",
    "annotation_type" : "polygon",
    "annotator" : "annotator1"}
    
}


{'Cat_0002_small_bowel#_hypergui_1': {'label_name': 'small_bowel',
  'annotation_type': 'polygon',
  'annotator': 'annotator1'},
 'Cat_0003_colon#_hypergui_1': {'label_name': 'colon',
  'annotation_type': 'polygon',
  'annotator': 'annotator1'}}

In this example, I have two datasets that I want to analyze together: Small bowel and Colon, but you will create your own to match your own experiment.

in this JSON, each unique type or organ/label gets its own mapping to a combination of a dataset and a hypergui number. This is because in some datasets, there are multiple Hypergui folders, e.g. _hypergui_1 and hypergui_2, where each annotates a different part of an organ. in such a case, each annotation type would get its own entry in the JSON file.

The dictionary key defining the mapping for each organ should always be in the format:

"<dataset_name>#<_hypergui_n>"

The value correspoding to that key is then itself a dictionary with the following keys:

### label_name:
Please give here the nam,e you want for the organ as a string.
NOTE: while it may work to map multiple dataset#hypergui directories to the same name, it is in your interest to provide a unique mapping for each directory. For example, if I have two directories that both annotate a kidney but in different conditions, e.g. perfused and malperfused, I will want to label each uniquely  with names like 'kidney_per' and 'kidney_malper'. This way the htc can tell the difference between them, EVEN IF you only intend to treat them both as kindey in the actual analysis. This is because we can remap them together as just 'kidney' when we actually come to the training section. In other words, be as specific as possible when creating the external dataset, because you can always get more general later. 

### annotation_type:
as of most recent update, the modified htc only works for polygon annotations, so please just leave this as polygon

### annotator:
this alows you to specify the name of the annotator(s) you want. for now, please just use annotator#1, as the author has not fully implemented multiple annotators yet.

Please create a file named 'subdata_hypergui_mapping.json' in your external directory, and write your own version according to the mappings you want to make (you can copy paste the axmple to easily maintain the formatting).

Finally, you are ready to generate your external data! in the cloned htc repository, navigate to htc/dataset_preparation/run_dataset.sh,  and open the file.  under input_paths, simply add strings of the absolute path to your datasets (the same path that you used in the .env file.) when you are done, simply run the run_dataset.sh from the root of the repository. you can do this with:

```bash
chmod -x htc/dataset_preparation/run_dataset.sh
sh htc/dataset_preparation/run_dataset.sh
```

Let the script run (it will take a little while), and when its done you are ready to train your network! you can move on to either the SegmentationTraining.ipynb or the TissueAtlassClassTraining.ipynb, depending on your goal.